In [1]:
import os
from Bio import SeqIO

In [2]:
def read_data():
    base_directory = './h_pylori_paidb//'
    data = {}
    
    for name in os.listdir(base_directory):
        dir_path = os.path.join(base_directory, name)

        if os.path.isdir(dir_path):
            fasta_file = os.path.join(dir_path, f"{name}.fasta")
            pai_file   = os.path.join(dir_path, f"{name}.pai")
            cpai_file  = os.path.join(dir_path, f"{name}.cpai")
            npai_file  = os.path.join(dir_path, f"{name}.npai")

            data[name] = {}
            genome_sequence = next(SeqIO.parse(fasta_file, 'fasta'))
            data[name]['genome_sequence'] = genome_sequence

            with open(pai_file, 'r') as file:
                data[name]['pai_sequences'] = []

                while True:
                    line = file.readline()
                    if not line:
                        break

                    line = line.split()
                    pai_start_index = int(line[0])
                    pai_end_index = int(line[1])

                    pai_sequence = genome_sequence.seq[pai_start_index:pai_end_index]

                    data[name]['pai_sequences'].append((pai_sequence, (pai_start_index, pai_end_index)))

            with open(cpai_file, 'r') as file:
                data[name]['cpai_sequences'] = []

                while True:
                    line = file.readline()
                    if not line:
                        break

                    line = line.split()
                    cpai_start_index = int(line[0])
                    cpai_end_index = int(line[1])

                    cpai_sequence = genome_sequence.seq[cpai_start_index:cpai_end_index]

                    data[name]['cpai_sequences'].append((cpai_sequence, (cpai_start_index, cpai_end_index)))

            with open(npai_file, 'r') as file:
                data[name]['npai_sequences'] = []

                while True:
                    line = file.readline()
                    if not line:
                        break

                    line = line.split()
                    npai_start_index = int(line[0])
                    npai_end_index = int(line[1])

                    npai_sequence = genome_sequence.seq[npai_start_index:npai_end_index]

                    data[name]['npai_sequences'].append((npai_sequence, (npai_start_index, npai_end_index)))

    return data

In [3]:
h_pylori_data = read_data()

### TF-IDF

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

def tfidf(sequence, k):
    kmers = get_kmers(str(sequence), k)
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform([kmers])
    tfidf_matrix = X.toarray()
    return tfidf_matrix, vectorizer

def get_kmers(sequence, k):
    kmers = [sequence[i:i+k] for i in range(len(sequence) - k + 1)]
    return ' '.join(kmers)

def get_kmers_map(tfidf_matrix, vectorizer, sequence, top_n=20):
    feature_names = vectorizer.get_feature_names_out()
    feature_scores = tfidf_matrix.flatten()
    
    top_kmers_indices = feature_scores.argsort()[-top_n:][::-1]
    top_kmers = [(feature_names[i], feature_scores[i]) for i in top_kmers_indices]

    mapped_kmers = {}
    for kmer, score in top_kmers:
        kmer = kmer.upper()
        count = sequence.count(kmer)
        mapped_kmers[kmer] = {'score': score, 'count': count}
        
    return mapped_kmers

def find_unique_kmers(full_sequence, start_index, end_index, significant_kmers, unique_kmers):
    for kmer, details in significant_kmers.items():
        left_occurrences = full_sequence[:start_index].count(kmer)
        right_occurrences = full_sequence[end_index:].count(kmer)
        occurrences = left_occurrences + right_occurrences
        if occurrences == 0:
            unique_kmers.append((kmer, details['count']))

In [5]:
def get_unique_patterns(full_sequence, island_sequence, start_index, end_index, k_max):
    
    unique_kmers = []
    
    for k in range(4, k_max):
        tfidf_matrix, vectorizer = tfidf(island_sequence, k)
        kmers = get_kmers_map(tfidf_matrix, vectorizer, island_sequence)
        find_unique_kmers(full_sequence, start_index, end_index, kmers, unique_kmers)
        
    return unique_kmers

In [6]:
import logging

logging.basicConfig(filename='./logs/tfidf_all_islands.log', level=logging.INFO,
                    format='%(message)s')

console = logging.StreamHandler()
console.setLevel(logging.INFO)
console.setFormatter(logging.Formatter('%(message)s'))

logging.getLogger('').addHandler(console)

In [6]:
from Bio import Align
from Bio.Seq import Seq

In [7]:
def count_matches(sequence, pattern):
#     aligner = Align.PairwiseAligner()
#     aligner.mode = 'local'
#     aligner.match_score = 2
#     aligner.mismatch_score = -2
#     aligner.open_gap_score = -2
#     aligner.extend_gap_score = -1
    
#     alignments = aligner.align(sequence, pattern)
#     threshold = 0.85 * (2 * len(pattern))
    
#     return sum(1 for alignment in alignments if alignment.score >= threshold)
    return sequence.count(pattern)

In [8]:
def find_islands_kmers(full_sequence,
                       pai_islands,
                       cpai_islands,
                       npai_islands,
                       significant_kmers,
                       all_islands_kmers):
    
    all_islands = pai_islands + cpai_islands + npai_islands
    all_islands.sort(key=lambda x: x[0])
        
    for kmer, details in significant_kmers.items():
        
        if kmer in all_islands_kmers:
            continue
        
        start = 0
        end = 0
        isUnique = True
        
        # Check if kmer doesn't exist outside of the islands
        for island in all_islands:
            end = island[0]
            
            if end > start:
                if count_matches(full_sequence[start:end], kmer) > 0:
                    isUnique = False
                    break
                    
            start = island[1]
                    
        if not isUnique or count_matches(full_sequence[start:], kmer) > 0:
            continue
          
        
        # Check which islands contain kmer
        islands_containing_kmer = []
        
        for num, pai_island in enumerate(pai_islands, start=1):
            start = pai_island[0]
            end = pai_island[1]
            
            count = count_matches(full_sequence[start:end], kmer)
            
            if count > 0:
                islands_containing_kmer.append(f'PAI {num}: {count}')
            
        for num, cpai_island in enumerate(cpai_islands, start=1):
            start = cpai_island[0]
            end = cpai_island[1]
            
            count = count_matches(full_sequence[start:end], kmer)
            
            if count > 0:
                islands_containing_kmer.append(f'CPAI {num}: {count}')
                
        for num, npai_island in enumerate(npai_islands, start=1):
            start = npai_island[0]
            end = npai_island[1]
            
            count = count_matches(full_sequence[start:end], kmer)
            
            if count > 0:
                islands_containing_kmer.append(f'NPAI {num}: {count}')
        
        all_islands_kmers[kmer] = islands_containing_kmer

In [9]:
def get_islands_kmers(full_sequence,
                      island_sequence,
                      pai_islands,
                      cpai_islands,
                      npai_islands,
                      all_islands_kmers,
                      k_max):
    
    
    for k in range(4, k_max):
        tfidf_matrix, vectorizer = tfidf(island_sequence, k)
        kmers = get_kmers_map(tfidf_matrix, vectorizer, island_sequence)
        find_islands_kmers(full_sequence, pai_islands, cpai_islands, npai_islands, kmers, all_islands_kmers)

In [11]:
k_max = 40
logging.info(f'Max pattern length: {k_max}')
logging.info('--------------------------')

unique_patterns = {}

for i, (name, data) in enumerate(h_pylori_data.items()):
    logging.info(f'Genome {i+1}/{len(h_pylori_data)}: {name}')
    
    genome_sequence = data['genome_sequence']
    all_islands_kmers = {}

    pai_indices = [x[1] for x in data['pai_sequences']]
    cpai_indices = [x[1] for x in data['cpai_sequences']]
    npai_indices = [x[1] for x in data['npai_sequences']]
    
    if len(data['pai_sequences']) != 0:
        
        for j, (pai_sequence, indices) in enumerate(data['pai_sequences']):
            
            len_before = len(all_islands_kmers)
            
            get_islands_kmers(genome_sequence,
                              pai_sequence,
                              pai_indices,
                              cpai_indices,
                              npai_indices,
                              all_islands_kmers,
                              k_max)
            
            new_patterns_found = len(all_islands_kmers) - len_before
            logging.info(f"PAI {j+1}/{len(data['pai_sequences'])}: {new_patterns_found} patterns found")
                        
    else:
        logging.info('No PAI sequences')
    
    
    if len(data['cpai_sequences']) != 0:
        
        for j, (cpai_sequence, indices) in enumerate(data['cpai_sequences']):
            
            len_before = len(all_islands_kmers)
            
            get_islands_kmers(genome_sequence,
                              cpai_sequence,
                              pai_indices,
                              cpai_indices,
                              npai_indices,
                              all_islands_kmers,
                              k_max)
            
            new_patterns_found = len(all_islands_kmers) - len_before
            logging.info(f"CPAI {j+1}/{len(data['cpai_sequences'])}: {new_patterns_found} patterns found")
            
    else:
        logging.info('No CPAI sequences')
        
        
    if len(data['npai_sequences']) != 0:

        for j, (npai_sequence, indices) in enumerate(data['npai_sequences']):
            
            len_before = len(all_islands_kmers)
            
            get_islands_kmers(genome_sequence,
                              npai_sequence,
                              pai_indices,
                              cpai_indices,
                              npai_indices,
                              all_islands_kmers,
                              k_max)
            
            new_patterns_found = len(all_islands_kmers) - len_before
            logging.info(f"NPAI {j+1}/{len(data['npai_sequences'])}: {new_patterns_found} patterns found")
    else:
        logging.info('No NPAI sequences')
    logging.info('--------------------------')
    
    unique_patterns[name] = all_islands_kmers
    os.makedirs(f'h_pylori_paidb/{name}/patterns/tfidf/all_islands/', exist_ok=True)
    
    with open(f'h_pylori_paidb/{name}/patterns/tfidf/all_islands/tfidf.txt', 'w') as file:
        for kmer in all_islands_kmers:
            file.write(kmer)
            file.write('\n')
            
            for island in all_islands_kmers[kmer]:
                file.write(island)
                file.write('\n')
            
            file.write('----------------\n')

Max pattern length: 40
--------------------------
Genome 1/53: NC_017372
No PAI sequences
CPAI 1/1: 538 patterns found
NPAI 1/1: 555 patterns found
--------------------------
Genome 2/53: NC_011333
No PAI sequences
No CPAI sequences
NPAI 1/2: 550 patterns found
NPAI 2/2: 533 patterns found
--------------------------
Genome 3/53: NC_008086
No PAI sequences
CPAI 1/1: 542 patterns found
No NPAI sequences
--------------------------
Genome 4/53: NC_021882
No PAI sequences
CPAI 1/1: 524 patterns found
No NPAI sequences
--------------------------
Genome 5/53: NC_010698
No PAI sequences
CPAI 1/1: 524 patterns found
NPAI 1/1: 550 patterns found
--------------------------
Genome 6/53: NC_017359
No PAI sequences
No CPAI sequences
NPAI 1/1: 552 patterns found
--------------------------
Genome 7/53: NC_017355
No PAI sequences
No CPAI sequences
NPAI 1/2: 550 patterns found
NPAI 2/2: 564 patterns found
--------------------------
Genome 8/53: NC_017375
No PAI sequences
CPAI 1/1: 557 patterns found
No 

In [13]:
import json

In [14]:
with open('./json/tfidf/all_islands.json', 'w') as file:
    json.dump(unique_patterns, file)

#### Patterns that partially belong in the islands

In [10]:
import logging

logging.basicConfig(filename='./logs/tfidf_partial.log', level=logging.INFO,
                    format='%(message)s')

console = logging.StreamHandler()
console.setLevel(logging.INFO)
console.setFormatter(logging.Formatter('%(message)s'))

logging.getLogger('').addHandler(console)

In [11]:
def find_unique_kmers_partial(full_sequence, island_start_index, island_end_index, significant_kmers, unique_kmers):
    for kmer, details in significant_kmers.items():
        
        is_unique = True
        is_partial = False
        index = 0
        
        while index != -1:
            index = full_sequence.seq.find(kmer, index)
            if index != -1:
                start = index
                end = index + len(kmer)
                
                if start < island_start_index:
                    if end < island_start_index:
                        is_unique = False
                        break
                    else:
                        is_partial = True
                        
                if end > island_end_index:
                    if start > island_end_index:
                        is_unique = False
                        break
                    else:
                        is_partial = True
                
                index += len(kmer)
                
        if not is_unique or not is_partial:
            continue
        
        left_occurrences = full_sequence[:island_start_index].count(kmer)
        right_occurrences = full_sequence[island_end_index:].count(kmer)
        occurrences = left_occurrences + right_occurrences
        
        if occurrences == 0:
            unique_kmers.append((kmer, details['count']))
        else:
            print('Something went wrong')
            
def get_unique_patterns_partial(full_sequence, 
                                island_start_index,
                                island_end_index,
                                k_max):
    
    unique_kmers = []
    
    for k in range(4, k_max):
        
        start = island_start_index-k+1
        end = island_end_index+k-1
        
        if start < 0:
            start = 0
            
        if end > len(full_sequence.seq) - 1:
            end = len(full_sequence.seq) - 1
        
        island_sequence = full_sequence.seq[start:end]
        
        tfidf_matrix, vectorizer = tfidf(island_sequence, k)
        kmers = get_kmers_map(tfidf_matrix, vectorizer, island_sequence)
        find_unique_kmers_partial(full_sequence, island_start_index, island_end_index, kmers, unique_kmers)
        
    return unique_kmers

In [12]:
unique_pai_patterns  = {}
unique_cpai_patterns = {}
unique_npai_patterns = {}

k_max = 40
logging.info(f'Max pattern length: {k_max}')
logging.info('--------------------------')

for i, (name, data) in enumerate(h_pylori_data.items()):        
    logging.info(f'Genome {i+1}/{len(h_pylori_data)}: {name}')
    
    if len(data['pai_sequences']) != 0:
        unique_pai_patterns[name] = {}
        genome_sequence = data['genome_sequence']
        for j, (pai_sequence, indices) in enumerate(data['pai_sequences']):
            unique_patterns = get_unique_patterns_partial(genome_sequence, indices[0], indices[1], k_max)

            unique_pai_patterns[name][f'pai_sequence_{j+1}'] = pai_sequence
            unique_pai_patterns[name][f'indices_pai_{j+1}'] = indices
            unique_pai_patterns[name][f'patterns_pai_{j+1}'] = unique_patterns

            logging.info(f"PAI {j+1}/{len(data['pai_sequences'])}: {len(unique_patterns)} unique patterns")
            
            os.makedirs(f'h_pylori_paidb/{name}/patterns/tfidf/partial/pai/', exist_ok=True)
            
            with open(f'h_pylori_paidb/{name}/patterns/tfidf/partial/pai/pai_{j+1}_patterns.txt', 'w') as file:
                for unique_pattern, frequency in unique_patterns:
                    file.write(f'{unique_pattern}: {frequency}\n')
                        
    else:
        logging.info('No PAI sequences')
    
    if len(data['cpai_sequences']) != 0:    
        unique_cpai_patterns[name] = {}

        genome_sequence = data['genome_sequence']
        for j, (cpai_sequence, indices) in enumerate(data['cpai_sequences']):
            unique_patterns = get_unique_patterns_partial(genome_sequence, indices[0], indices[1], k_max)

            unique_cpai_patterns[name][f'cpai_sequence_{j+1}'] = cpai_sequence
            unique_cpai_patterns[name][f'indices_cpai_{j+1}'] = indices
            unique_cpai_patterns[name][f'patterns_cpai_{j+1}'] = unique_patterns
            
            logging.info(f"CPAI {j+1}/{len(data['cpai_sequences'])}: {len(unique_patterns)} unique patterns")
            
            os.makedirs(f'h_pylori_paidb/{name}/patterns/tfidf/partial/cpai/', exist_ok=True)
            
            with open(f'h_pylori_paidb/{name}/patterns/tfidf/partial/cpai/cpai_{j+1}_patterns.txt', 'w') as file:
                for unique_pattern, frequency in unique_patterns:
                    file.write(f'{unique_pattern}: {frequency}\n')
            
    else:
        logging.info('No CPAI sequences')
        
    if len(data['npai_sequences']) != 0:
        unique_npai_patterns[name] = {}
        genome_sequence = data['genome_sequence']
        for j, (npai_sequence, indices) in enumerate(data['npai_sequences']):
            unique_patterns = get_unique_patterns_partial(genome_sequence, indices[0], indices[1], k_max)

            unique_npai_patterns[name][f'npai_sequence_{j+1}'] = npai_sequence
            unique_npai_patterns[name][f'indices_npai_{j+1}'] = indices
            unique_npai_patterns[name][f'patterns_npai_{j+1}'] = unique_patterns

            logging.info(f"NPAI {j+1}/{len(data['npai_sequences'])}: {len(unique_patterns)} unique patterns")
            
            os.makedirs(f'h_pylori_paidb/{name}/patterns/tfidf/partial/npai/', exist_ok=True)
            
            with open(f'h_pylori_paidb/{name}/patterns/tfidf/partial/npai/npai_{j+1}_patterns.txt', 'w') as file:
                for unique_pattern, frequency in unique_patterns:
                    file.write(f'{unique_pattern}: {frequency}\n')
    
    else:
        logging.info('No NPAI sequences')
    logging.info('--------------------------')

Max pattern length: 40
--------------------------
Genome 1/53: NC_017372
No PAI sequences
CPAI 1/1: 0 unique patterns
NPAI 1/1: 0 unique patterns
--------------------------
Genome 2/53: NC_011333
No PAI sequences
No CPAI sequences
NPAI 1/2: 0 unique patterns
NPAI 2/2: 0 unique patterns
--------------------------
Genome 3/53: NC_008086
No PAI sequences
CPAI 1/1: 0 unique patterns
No NPAI sequences
--------------------------
Genome 4/53: NC_021882
No PAI sequences
CPAI 1/1: 0 unique patterns
No NPAI sequences
--------------------------
Genome 5/53: NC_010698
No PAI sequences
CPAI 1/1: 0 unique patterns
NPAI 1/1: 0 unique patterns
--------------------------
Genome 6/53: NC_017359
No PAI sequences
No CPAI sequences
NPAI 1/1: 0 unique patterns
--------------------------
Genome 7/53: NC_017355
No PAI sequences
No CPAI sequences
NPAI 1/2: 0 unique patterns
NPAI 2/2: 2 unique patterns
--------------------------
Genome 8/53: NC_017375
No PAI sequences
CPAI 1/1: 0 unique patterns
No NPAI sequenc

### NOSEP

In [4]:
from spmf import Spmf
import os
from Bio import SeqIO

In [5]:
def to_spmf_format(genome_sequence):
    seq = str(genome_sequence)
    
    seq = seq.replace('A', '1')
    seq = seq.replace('C', '2')
    seq = seq.replace('G', '3')
    seq = seq.replace('T', '4')

    seq = ''.join(c + ' -1 ' for c in seq).rstrip(' -1 ') + ' -2'
    
    return seq

In [6]:
def from_spmf_format(filename):
    patterns = []
    
    with open(filename, 'r') as file:
        for line in file:
            lst = line.split('#')
            
            pattern = lst[0]
            support = lst[1]
            
            pattern = pattern.replace('-1', '')
            pattern = pattern.replace('1', 'A')
            pattern = pattern.replace('2', 'C')
            pattern = pattern.replace('3', 'G')
            pattern = pattern.replace('4', 'T')
            pattern = pattern.replace(' ', '')
            
            support = int(support.replace('SUP: ', ''))
            patterns.append((pattern, support))
            
    return patterns

In [7]:
import logging

logging.basicConfig(filename='logs/nosep.log', level=logging.INFO,
                    format='%(message)s')

console = logging.StreamHandler()
console.setLevel(logging.INFO)
console.setFormatter(logging.Formatter('%(message)s'))

logging.getLogger('').addHandler(console)

In [8]:
def run_nosep(full_sequence, island_sequence, start_index, end_index, k_max):
    
    unique_patterns = []
    
    input_filename = 'input.txt'
    output_filename = 'output.txt'
    
    with open(input_filename, 'w') as input_file:
        input_file.write(to_spmf_format(island_sequence))
    
    spmf = Spmf("NOSEP", input_filename=input_filename, output_filename=output_filename, arguments=[4, k_max, 0, 0, 2])
    spmf.run()
    
    patterns = from_spmf_format(output_filename)
    
    os.remove(input_filename)
    os.remove(output_filename)
    
    for pattern, support in patterns:
        left_occurrences = full_sequence[:start_index].count(pattern)
        right_occurrences = full_sequence[end_index:].count(pattern)
        occurrences = left_occurrences + right_occurrences
        if occurrences == 0:
            unique_patterns.append((pattern, support))
            
    return unique_patterns

In [9]:
unique_pai_patterns  = {}
unique_cpai_patterns = {}
unique_npai_patterns = {}

k_max = 40
logging.info(f'Max pattern length: {k_max}')
logging.info('--------------------------')

for i, (name, data) in enumerate(h_pylori_data.items()):
    logging.info(f'Genome {i+1}/{len(h_pylori_data)}: {name}')
    
    if len(data['pai_sequences']) != 0:
        unique_pai_patterns[name] = {}
        genome_sequence = data['genome_sequence']
        for j, (pai_sequence, indices) in enumerate(data['pai_sequences']):
            
            if indices[1] - indices[0] > 40000:
                logging.info(f"PAI {j+1}/{len(data['pai_sequences'])}: island too large")
                continue
                
            unique_patterns = run_nosep(genome_sequence, pai_sequence, indices[0], indices[1], k_max)

            unique_pai_patterns[name]['pai_sequence'] = pai_sequence
            unique_pai_patterns[name]['indices'] = indices
            unique_pai_patterns[name]['patterns'] = unique_patterns

            logging.info(f"PAI {j+1}/{len(data['pai_sequences'])}: {len(unique_patterns)} unique patterns")
            
            os.makedirs(f'h_pylori_paidb/{name}/patterns/nosep/pai/', exist_ok=True)
            
            with open(f'h_pylori_paidb/{name}/patterns/nosep/pai/pai_{j}_patterns.txt', 'w') as file:
                for unique_pattern, frequency in unique_patterns:
                    file.write(f'{unique_pattern}: {frequency}\n')
                        
    else:
        logging.info('No PAI sequences')
    
    if len(data['cpai_sequences']) != 0:    
        unique_cpai_patterns[name] = {}

        genome_sequence = data['genome_sequence']
        for j, (cpai_sequence, indices) in enumerate(data['cpai_sequences']):
            
            if indices[1] - indices[0] > 40000:
                logging.info(f"CPAI {j+1}/{len(data['cpai_sequences'])}: island too large")
                continue
            
            unique_patterns = run_nosep(genome_sequence, cpai_sequence, indices[0], indices[1], k_max)

            unique_cpai_patterns[name]['cpai_sequence'] = cpai_sequence
            unique_cpai_patterns[name]['indices'] = indices
            unique_cpai_patterns[name]['patterns'] = unique_patterns
            
            logging.info(f"CPAI {j+1}/{len(data['cpai_sequences'])}: {len(unique_patterns)} unique patterns")
            
            os.makedirs(f'h_pylori_paidb/{name}/patterns/nosep/cpai/', exist_ok=True)
            
            with open(f'h_pylori_paidb/{name}/patterns/nosep/cpai/cpai_{j}_patterns.txt', 'w') as file:
                for unique_pattern, frequency in unique_patterns:
                    file.write(f'{unique_pattern}: {frequency}\n')
            
    else:
        logging.info('No CPAI sequences')
        
    if len(data['npai_sequences']) != 0:
        unique_npai_patterns[name] = {}
        genome_sequence = data['genome_sequence']
        for j, (npai_sequence, indices) in enumerate(data['npai_sequences']):
            
            if indices[1] - indices[0] > 40000:
                logging.info(f"NPAI {j+1}/{len(data['npai_sequences'])}: island too large")
                continue
            
            unique_patterns = run_nosep(genome_sequence, npai_sequence, indices[0], indices[1], k_max)

            unique_npai_patterns[name]['npai_sequence'] = npai_sequence
            unique_npai_patterns[name]['indices'] = indices
            unique_npai_patterns[name]['patterns'] = unique_patterns

            logging.info(f"NPAI {j+1}/{len(data['npai_sequences'])}: {len(unique_patterns)} unique patterns")
            
            os.makedirs(f'h_pylori_paidb/{name}/patterns/nosep/npai/', exist_ok=True)
            
            with open(f'h_pylori_paidb/{name}/patterns/nosep/npai/npai_{j}_patterns.txt', 'w') as file:
                for unique_pattern, frequency in unique_patterns:
                    file.write(f'{unique_pattern}: {frequency}\n')
    
    else:
        logging.info('No NPAI sequences')
    logging.info('--------------------------')

Max pattern length: 40
--------------------------
Genome 1/53: NC_017372
No PAI sequences


>/home/maja/.local/lib/python3.10/site-packages/spmf/spmf.jar
=============  NOSEP v2.46 - STATS =============
 Number of patterns found: 13971
 Total time ~ 34880 ms
 Maximum memory usage : 294.0252990722656 mb
 Calculation count 27737



CPAI 1/1: 314 unique patterns
NPAI 1/1: island too large
--------------------------
Genome 2/53: NC_011333
No PAI sequences
No CPAI sequences
NPAI 1/2: island too large


>/home/maja/.local/lib/python3.10/site-packages/spmf/spmf.jar
=============  NOSEP v2.46 - STATS =============
 Number of patterns found: 29246
 Total time ~ 176465 ms
 Maximum memory usage : 42.65605163574219 mb
 Calculation count 58527



NPAI 2/2: 594 unique patterns
--------------------------
Genome 3/53: NC_008086
No PAI sequences
CPAI 1/1: island too large
No NPAI sequences
--------------------------
Genome 4/53: NC_021882
No PAI sequences
CPAI 1/1: island too large
No NPAI sequences
--------------------------
Genome 5/53: NC_010698
No PAI sequences


...

CPAI 1/2: 494 unique patterns
CPAI 2/2: island too large


>/home/maja/.local/lib/python3.10/site-packages/spmf/spmf.jar
=============  NOSEP v2.46 - STATS =============
 Number of patterns found: 9094
 Total time ~ 17542 ms
 Maximum memory usage : 214.18203735351562 mb
 Calculation count 18565



NPAI 1/1: 66 unique patterns
--------------------------
Genome 48/53: NC_017381
No PAI sequences
No CPAI sequences
NPAI 1/1: island too large
--------------------------
Genome 49/53: NC_017362
No PAI sequences
No CPAI sequences
NPAI 1/1: island too large
--------------------------
Genome 50/53: NC_021215
No PAI sequences
CPAI 1/1: island too large
No NPAI sequences
--------------------------
Genome 51/53: NC_017360
No PAI sequences
CPAI 1/1: island too large
No NPAI sequences
--------------------------
Genome 52/53: NC_017365


>/home/maja/.local/lib/python3.10/site-packages/spmf/spmf.jar
=============  NOSEP v2.46 - STATS =============
 Number of patterns found: 58019
 Total time ~ 349504 ms
 Maximum memory usage : 159.26554107666016 mb
 Calculation count 88015



PAI 1/1: 28634 unique patterns
CPAI 1/1: island too large
No NPAI sequences
--------------------------
Genome 53/53: NC_017354
No PAI sequences
No CPAI sequences
NPAI 1/1: island too large
--------------------------


In [18]:
with open('./json/nosep/pai_patterns.json', 'w') as file:
    json.dump(unique_pai_patterns, file)
    
with open('./json/nosep/cpai_patterns.json', 'w') as file:
    json.dump(unique_cpai_patterns, file)
    
with open('./json/nosep/npai_patterns.json', 'w') as file:
    json.dump(unique_npai_patterns, file)